In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import datetime, time, gc, pickle
from scipy.stats import skew, kurtosis
# from catboost import CatBoostClassifier
from zipfile import ZipFile
from collections import Counter 
from sklearn.metrics import roc_auc_score as auc
# import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
from tensorflow.keras.callbacks import * 
import tensorflow as tf 

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

from deepctr.models import * 
from deepctr.inputs import SparseFeat, DenseFeat, get_feature_names

In [2]:
!free -m 

              total        used        free      shared  buff/cache   available
Mem:          80041       20046       39356          18       20638       59843
Swap:             0           0           0


DeepCTR version 0.8.1 detected. Your version is 0.7.4.
Use `pip install -U deepctr` to upgrade.Changelog: https://github.com/shenweichen/DeepCTR/releases/tag/v0.8.1


In [3]:
data = pd.read_pickle('data/train_test_fea.pkl')
print(data.shape)
data = pd.concat([data, pd.read_pickle("data/train_test_countfea.pkl")], axis=1)
print(data.shape)

def reduce_mem_usage(df):
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in tqdm(df.columns):
        col_type = df[col].dtypes
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem,
                                                                          100 * (start_mem - end_mem) / start_mem))
    return df


sparse_features = [
       'task_id', 'adv_id', 'creat_type_cd', 'adv_prim_id',
       'dev_id', 'inter_type_cd', 'slot_id', 'spread_app_id', 'tags',
       'app_first_class', 'app_second_class', 'city', 'age', 
       'device_name', 'career', 'gender', 'net_type',
       'residence', 'his_on_shelf_time', "communication_onlinerate", 
       'up_membership_grade', 'consume_purchase', 'indu_name',
]

drop_fea = [col for col in data.columns.tolist() if 'ctr' in col or 'target_enc' in col] # 
print(len(drop_fea))

dense_features = list(set(data.columns) - set(sparse_features + ['label', 'cnt', 'random_sector', 'pt_d', 'id',
        'uid_task_id_ctr', 'uid_adv_id_ctr', 'uid_spread_app_id_ctr', 'uid_adv_prim_id_ctr', 'uid_dev_id_ctr',
       'uid_app_second_class_ctr'] + drop_fea))
# use_cols = list(set(train.columns) - set(['label']))
print(len(sparse_features), len(dense_features))

(8000000, 148)
(8000000, 195)
0
23 170


In [4]:
%%time

# data[sparse_features] = data[sparse_features].fillna(0,)
data[dense_features] = data[dense_features].fillna(0,)
target = ['label']

for c in tqdm(dense_features):
#     MAX = data[c].max()
#     MIN = data[c].min()
#     data[c] = (data[c] - MIN) / (MAX - MIN + 1e-8)
    mu = np.mean(data[c].values, axis=0)
    sigma = np.std(data[c].values, axis=0) + 1e-8
    data[c] = (data[c] - mu) / sigma

data = reduce_mem_usage(data)

for feat in tqdm(sparse_features):
    lbe = LabelEncoder()
    data[feat] = lbe.fit_transform(data[feat])
# mms = MinMaxScaler(feature_range=(0,1))
# data[dense_features] = mms.fit_transform(data[dense_features])

data = reduce_mem_usage(data)

gc.collect()

  2%|▏         | 3/170 [00:02<01:46,  1.57it/s]/opt/conda/envs/tensorflow_py3/lib/python3.6/site-packages/numpy/core/_methods.py:183: RuntimeWarning: overflow encountered in reduce
  arrmean = umr_sum(arr, axis, dtype, keepdims=True)
  0%|          | 0/23 [00:00<?, ?it/s]

Mem. usage decreased to 2929.69 Mb (64.6% reduction)


100%|██████████| 195/195 [00:41<00:00,  4.70it/s]

Mem. usage decreased to 2876.28 Mb (29.4% reduction)
CPU times: user 2min 8s, sys: 49.4 s, total: 2min 57s
Wall time: 2min 57s


22

In [5]:
fixlen_feature_columns = [SparseFeat(feat, vocabulary_size=data[feat].nunique(), embedding_dim=16)
                           for i,feat in enumerate(sparse_features)] + [DenseFeat(feat, 1,) for feat in dense_features]

dnn_feature_columns = fixlen_feature_columns
linear_feature_columns = fixlen_feature_columns

feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

In [6]:
# oof_train = np.zeros((7000000, 7)) 
# oof_train[:, 1][data.iloc[:-1000000].pt_d == 1] = np.random.randn(1000000, 1).reshape(-1)
# # oof_train, data.iloc[:-1000000].shape
# oof_train

In [7]:
# !pip install keras==2.2.4

file_name = datetime.date.today().strftime('%m%d')+"_{}.log".format("deep_base")
def write_log(w):
    t0 = datetime.datetime.now().strftime('%H:%M:%S')
    info = "{} : {}\n".format(t0, w)
    print(info)
    with open(file_name, 'a') as f:
        f.write(info)
        f.write("-"*80+"\n")


# def multi_category_focal_loss2(gamma=2., alpha=.25):
#     """
#     focal loss for multi category of multi label problem
#     适用于多分类或多标签问题的focal loss
#     alpha控制真值y_true为1/0时的权重
#         1的权重为alpha, 0的权重为1-alpha
#     当你的模型欠拟合，学习存在困难时，可以尝试适用本函数作为loss
#     当模型过于激进(无论何时总是倾向于预测出1),尝试将alpha调小
#     当模型过于惰性(无论何时总是倾向于预测出0,或是某一个固定的常数,说明没有学到有效特征)
#         尝试将alpha调大,鼓励模型进行预测出1。
#     Usage:
#      model.compile(loss=[multi_category_focal_loss2(alpha=0.25, gamma=2)], metrics=["accuracy"], optimizer=adam)
#     """
#     epsilon = 1.e-7
#     gamma = float(gamma)
#     alpha = tf.constant(alpha, dtype=tf.float32)

#     def multi_category_focal_loss2_fixed(y_true, y_pred):
#         y_true = tf.cast(y_true, tf.float32)
#         y_pred = tf.clip_by_value(y_pred, epsilon, 1. - epsilon)

#         alpha_t = y_true * alpha + (tf.ones_like(y_true) - y_true) * (1 - alpha)
#         y_t = tf.multiply(y_true, y_pred) + tf.multiply(1 - y_true, 1 - y_pred)
#         ce = -tf.math.log(y_t)
#         weight = tf.pow(tf.subtract(1., y_t), gamma)
#         fl = tf.multiply(tf.multiply(weight, ce), alpha_t)
#         loss = tf.reduce_mean(fl)
#         return loss

#     return multi_category_focal_loss2_fixed

# def auroc(y_true, y_pred):
#     return tf.compat.v1.py_func(roc_auc_score, (y_true, y_pred), tf.double)


test_pred = np.zeros((1000000, 7))
oof_train = np.zeros((7000000, 1))

for i in range(1, 8):
    write_log('pt_d = {}'.format(i))
    
    train = data.iloc[:-1000000][data["pt_d"] != i].reset_index(drop=True)
    valid = data.iloc[:-1000000][data["pt_d"] == i].reset_index(drop=True)
    test = data.iloc[-1000000:].reset_index(drop=True)

    print(train.shape, valid.shape, test.shape)

    train_model_input = {name:train[name].values for name in feature_names}
    valid_model_input = {name:valid[name].values for name in feature_names}
    test_model_input = {name:test[name].values for name in feature_names}

    train_label = train['label'].values
    valid_label = valid['label'].values
    print(train_label.shape, valid_label.shape)

    del train, valid, test
    gc.collect()
    
    model = DeepFM(linear_feature_columns, dnn_feature_columns,task='binary', 
                   dnn_dropout=0.1, dnn_hidden_units=(512, 128))
#     model = FiBiNET(linear_feature_columns,dnn_feature_columns, task='binary',
#                     dnn_dropout=0.2, dnn_hidden_units=(512, 256),)
#     model = xDeepFM(linear_feature_columns,dnn_feature_columns, task='binary',
#                     dnn_dropout=0.2, dnn_hidden_units=(512, 128),)
#     model = AutoInt(linear_feature_columns,dnn_feature_columns, task='binary', 
#                     dnn_use_bn=True,
#                     dnn_dropout=0.1, dnn_hidden_units=(512, 128),)
    # model = FLEN(linear_feature_columns, dnn_feature_columns, task='binary',
    #             dnn_dropout=0.2, dnn_hidden_units=(512, 128),)
    
    model.compile("adam", 'binary_crossentropy',
                  metrics=['binary_crossentropy'])
    
    class Evaluate(Callback):
        def __init__(self):
            self.best_auc = 0.0 
         
        def on_epoch_end(self, epoch, logs=None):
            score = self.evaluate()
            if score > self.best_auc:
                self.best_auc = score
                model.save_weights('data/best_model_for_ctr.weights')
            print('current auc: %.6f, best auc: %.6f\n' % (score, self.best_auc))
            write_log('current auc: %.6f, best auc: %.6f\n' % (score, self.best_auc))
            
        def evaluate(self):
            val_pred = model.predict(valid_model_input, verbose=1, batch_size=4096)   # [bs, 10]
            return auc(valid_label, val_pred)

    evaluator = Evaluate()
    history = model.fit(train_model_input, train_label,
                        batch_size=4096*4, epochs=20, verbose=1, callbacks=[evaluator])
    model.load_weights("data/best_model_for_ctr.weights")
    pred_ans1 = model.predict(test_model_input, batch_size=4096, verbose=1)
    oof_train[data.iloc[:-1000000].pt_d == i] = model.predict(valid_model_input, batch_size=4096, verbose=1)
    test_pred[:, i-1] = pred_ans1.reshape(-1)

16:02:24 : pt_d = 1



/opt/conda/envs/tensorflow_py3/lib/python3.6/site-packages/ipykernel/__main__.py:54: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/opt/conda/envs/tensorflow_py3/lib/python3.6/site-packages/ipykernel/__main__.py:55: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


(6000000, 195) (1000000, 195) (1000000, 195)
(6000000,) (1000000,)
Epoch 1/20


/opt/conda/envs/tensorflow_py3/lib/python3.6/site-packages/tensorflow/python/framework/indexed_slices.py:432: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


245/245 [==============================] - 4s 15ms/step
current auc: 0.812775, best auc: 0.812775

16:03:48 : current auc: 0.812775, best auc: 0.812775


367/367 [==============================] - 50s 136ms/step - loss: 0.1307 - binary_crossentropy: 0.1305
Epoch 2/20
245/245 [==============================] - 4s 15ms/step
current auc: 0.819118, best auc: 0.819118

16:04:37 : current auc: 0.819118, best auc: 0.819118


367/367 [==============================] - 49s 134ms/step - loss: 0.1225 - binary_crossentropy: 0.1222
Epoch 3/20
245/245 [==============================] - 4s 15ms/step
current auc: 0.821698, best auc: 0.821698

16:07:04 : current auc: 0.821698, best auc: 0.821698


367/367 [==============================] - 49s 133ms/step - loss: 0.1207 - binary_crossentropy: 0.1203
Epoch 6/20
245/245 [==============================] - 4s 15ms/step
current auc: 0.823321, best auc: 0.823321

16:07:54 : current auc: 0.823321, best auc: 0.823321


367/367 [==============================] -

...

(6000000, 195) (1000000, 195) (1000000, 195)
(6000000,) (1000000,)
Epoch 1/20


/opt/conda/envs/tensorflow_py3/lib/python3.6/site-packages/tensorflow/python/framework/indexed_slices.py:432: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


245/245 [==============================] - 4s 14ms/step
current auc: 0.798602, best auc: 0.798602

17:46:22 : current auc: 0.798602, best auc: 0.798602


367/367 [==============================] - 48s 132ms/step - loss: 0.1330 - binary_crossentropy: 0.1328
Epoch 2/20
245/245 [==============================] - 4s 15ms/step
current auc: 0.801894, best auc: 0.801894

17:47:10 : current auc: 0.801894, best auc: 0.801894


367/367 [==============================] - 49s 133ms/step - loss: 0.1230 - binary_crossentropy: 0.1227
Epoch 3/20
245/245 [==============================] - 4s 15ms/step
current auc: 0.803290, best auc: 0.803290

17:48:00 : current auc: 0.803290, best auc: 0.803290


367/367 [==============================] - 50s 135ms/step - loss: 0.1221 - binary_crossentropy: 0.1217
Epoch 4/20
245/245 [==============================] - 4s 14ms/step
current auc: 0.801441, best auc: 0.803290

17:48:50 : current auc: 0.801441, best auc: 0.803290


367/367 [==============================] -

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



245/245 [==============================] - 4s 16ms/step
current auc: 0.803863, best auc: 0.804254

17:51:19 : current auc: 0.803863, best auc: 0.804254


367/367 [==============================] - 50s 136ms/step - loss: 0.1207 - binary_crossentropy: 0.1202
Epoch 8/20
245/245 [==============================] - 4s 16ms/step
current auc: 0.802293, best auc: 0.804254

17:52:09 : current auc: 0.802293, best auc: 0.804254


367/367 [==============================] - 50s 135ms/step - loss: 0.1205 - binary_crossentropy: 0.1200
Epoch 9/20
245/245 [==============================] - 4s 15ms/step
current auc: 0.803377, best auc: 0.804254

17:52:58 : current auc: 0.803377, best auc: 0.804254


367/367 [==============================] - 49s 134ms/step - loss: 0.1203 - binary_crossentropy: 0.1198
Epoch 10/20
245/245 [==============================] - 3s 14ms/step
current auc: 0.802855, best auc: 0.804254

17:53:48 : current auc: 0.802855, best auc: 0.804254


367/367 [==============================] 

In [19]:
test_pred

array([[0.01259109, 0.01506275, 0.01335475, ..., 0.01426241, 0.01060739,
        0.01720661],
       [0.04197159, 0.04554772, 0.08565068, ..., 0.0798614 , 0.07462114,
        0.04314351],
       [0.07388905, 0.05475152, 0.05977964, ..., 0.0621902 , 0.11411864,
        0.08130863],
       ...,
       [0.04704171, 0.03016657, 0.05516309, ..., 0.02802894, 0.03772154,
        0.03521585],
       [0.00917396, 0.01454204, 0.0237349 , ..., 0.00951687, 0.01083496,
        0.02637413],
       [0.02166238, 0.0138016 , 0.02376735, ..., 0.00921166, 0.01769921,
        0.01335332]])

In [20]:
test_res = (test_pred[:, 0]*0.1 + test_pred[:, 1]*0.1 + test_pred[:, 2]*0.1 + test_pred[:, 3]*0.1 + 
            test_pred[:, 4]*0.1 + test_pred[:, 5]*0.2 + test_pred[:, 6]*0.3)
test_res 

array([0.01406793, 0.05831028, 0.07922108, ..., 0.03861231, 0.0189604 ,
       0.015975  ])

In [16]:
test_pred.mean(-1)

array([0.01366553, 0.06031351, 0.07363927, ..., 0.03970998, 0.01800297,
       0.01647774])

In [14]:
np.save("oof_data/oof_train_autoint.npy", oof_train)
np.save("oof_data/oof_test_autoint.npy", test_pred)

In [21]:
result = pd.DataFrame({
        'id': np.load("data/test_id.npy"), 
        'probability': test_res, 
    })

result.to_csv("submission.csv", index=False) 

In [20]:
test_pred 

array([[0.01060276],
       [0.05616426],
       [0.10304786],
       ...,
       [0.03164182],
       [0.01153047],
       [0.01610501]])

In [2]:
import pandas as pd 
res1 = pd.read_csv("submission.csv")
res2 = pd.read_csv("submission_fibinet7787.csv")

res1['probability'] = res1['probability'] * 0.3 + res2['probability'] * 0.7 

In [3]:
res1.to_csv("submission.csv", index=False)